In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
ticker_symbol = 'AAPL'  # Substitua 'AAPL' pelo ticker do ativo financeiro desejado
start_date = '2022-01-01'  # Defina a data de início do histórico de dados
end_date = '2023-01-01'  # Defina a data de término do histórico de dados
moving_average_window = 20  # Janela da média móvel (20 dias neste exemplo)
num_standard_deviations = 2.5  # Número de desvios padrão para a banda superior e inferior
stop_loss_percent = 0.05  # Valor percentual para o stop loss (5% neste exemplo)

In [ ]:
data = yf.download(ticker, start=start_date, end=end_date)

In [ ]:
data['Close'].rolling(window=window).mean()

In [ ]:
data['Close'].rolling(window=window).std()

In [ ]:
data['SMA'] = calculate_moving_average(data, ma_window)
data['Upper_Band'] = data['SMA'] + (data['Close'].rolling(window=ma_window).std() * num_std)
data['Lower_Band'] = data['SMA'] - (data['Close'].rolling(window=ma_window).std() * num_std)
data['Signal'] = 0
data['Position'] = 0

In [ ]:
def calculate_cumulative_returns(data):
    data['Returns'] = data['Close'].pct_change()
    data['Position'] = 0  # Inicializa a coluna 'Position'
    data_with_positions = trading_strategy(data, moving_average_window, num_standard_deviations, stop_loss_percent)
    data['Cumulative_Returns'] = (data_with_positions['Returns'] * data_with_positions['Position']).cumsum()
    return data['Cumulative_Returns']

data['Cumulative_Returns'] = calculate_cumulative_returns(data)